# สมาชิกผู้จัดทำ

สิทธิเจตน์ วงศ์ทิชาวัฒน์ 6210503853

นทวัจน์ เมี้ยนละม้าย 6210503624

In [ ]:
! pip install mpi4py

In [ ]:
import numpy as np
import time
# Program to multiply two matrices using nested loops

t1 = time.time()
# 3x3 matrix
X = np.random.rand(100,100)
# 3x4 matrix
Y = np.random.rand(100,100)
# result is 3x4
result = np.zeros((100,100))

# iterate through rows of X
for i in range(len(X)):
   # iterate through columns of Y
   for j in range(len(Y[0])):
       # iterate through rows of Y
       for k in range(len(Y)):
           result[i][j] += X[i][k] * Y[k][j]
t2 = time.time()

print(t2-t1)
# for r in result:
#    print(r)

KeyboardInterrupt: ignored

# FILE : mpi-mul.py

ตัวแปร

**Line 11 :** workers คือ slave processes = comm.Get_size() - 1

**Line 14 :** N คือขนาดของ array NxN numpy array

**Line 17 :** mtrx_A คือ first numpy array value between 0 - 9 size NxN

**Line 18 :** mtrx_B คือ second numpy array value between 0 - 9 size NxN

**Line 19 :** mtrx_C คือ result numpy array

**Line 22 :** mul_matrix function คือ matrix multiplier function ทำ *Z[i][j] += X[i][k] x Y[k][j]* ในทุก slave process

**Line 30 :** distribute_data function คือแยก data ของ matrix จาก master process เป็นจำนวนเท่า workers และยัดเข้าไปใน slave process แต่ละตัว โดยใช้ comm.send()

**Line 54 :** assemble_data function คือรวม data ของ matrix ที่ผ่านการคูณมาแล้วใน slave processes มาใส่ใน mtrx_C โดยใช้ comm.recv ใน master process

จากผลที่ได้เวลาจาก 4, 8, 16 process จะน้อยลงตามลำดับแต่ก็ยังนานกว่า แบบ sequential อยู่ดี คิดว่าน่าจะมาจากการทำ matrix multiplication ต้องมีการสื่อสารมากทำให้ overhead เยอะมาก

In [ ]:
%%writefile mpi-mul.py
from mpi4py import MPI
from random import randint
import numpy as np
import time

comm = MPI.COMM_WORLD
rank = comm.Get_rank()
# comm.Get_size() = master_process + slave_processes
# workers = slave_processes
workers = comm.Get_size() - 1

# array of size 100x100, 1000x1000,5000x5000
N = 1000 # N = 100, 1000, 5000

# Generate numpy array NxN
mtrx_A = np.random.randint(10,size=(N,N))
mtrx_B = np.random.randint(10,size=(N,N))
mtrx_C = []


def mul_matrix(X, Y):
    # mul_matrix Z[i][j] += X[i][k] * Y[k][j]
    Z = [[sum(a * b for a, b in zip(X_row, Y_col)) for Y_col in zip(*Y)]
            for X_row in X]

    return Z


def distribute_data():
    # split matrix to p parts
    def split_matrix(seq, p):
        rows = []
        n = int(len(seq) / p)
        r = len(seq) % p
        b, e = 0, n + min(1, r)
        for i in range(p):
            rows.append(seq[b:e])
            r = max(0, r - 1)
            b, e = e, e + n + min(1, r)

        return rows

    rows = split_matrix(mtrx_A, workers)

    pid = 1
    for row in rows:
        # send p part of splited matrix to slave processes
        comm.send(row, dest=pid, tag=1)
        comm.send(mtrx_B, dest=pid, tag=2)
        pid = pid + 1


def assemble_data():
    global mtrx_C

    pid = 1
    for n in range(workers):
        # recieve p part of splited matrix from slave processes
        row = comm.recv(source=pid, tag=pid)
        # and assemble it to mtrx_C
        mtrx_C = mtrx_C + row
        pid = pid + 1


def master_process():
    # distribute matrix data to slaves
    distribute_data()
    # assemble salves returning values and generate final matrix
    assemble_data()


def slave_process():
    # receive data from master node
    x = comm.recv(source=0, tag=1)
    y = comm.recv(source=0, tag=2)

    # multiply the received matrix
    z = mul_matrix(x, y)
    # send the result back to master
    comm.send(z, dest=0, tag=rank)


if __name__ == '__main__':
    if rank == 0:
        # start time
        t1 = time.time()

        master_process()

        # end time
        t2 = time.time()

        # print(mtrx_A)
        # print(mtrx_B)
        # print(np.array(mtrx_C))
        print(f'Time : {t2 - t1} (s)')
    else:
        slave_process()

Overwriting mpi-mul.py


In [ ]:
# process include master_process 5, 9, 17 processes
# N = 100
! mpiexec --allow-run-as-root -np 5 python mpi-mul.py

In [ ]:
# process include master_process 5, 9, 17 processes
# N = 100
! mpiexec --allow-run-as-root -np 9 python mpi-mul.py

In [ ]:
# process include master_process 5, 9, 17 processes
# N = 100
! mpiexec --allow-run-as-root -np 17 python mpi-mul.py

[warn] Epoll ADD(4) on fd 29 failed.  Old events were 0; read change was 0 (none); write change was 1 (add): Bad file descriptor
[warn] Epoll ADD(4) on fd 71 failed.  Old events were 0; read change was 0 (none); write change was 1 (add): Bad file descriptor
[warn] Epoll ADD(4) on fd 74 failed.  Old events were 0; read change was 0 (none); write change was 1 (add): Bad file descriptor
[warn] Epoll ADD(4) on fd 83 failed.  Old events were 0; read change was 0 (none); write change was 1 (add): Bad file descriptor
[warn] Epoll ADD(4) on fd 84 failed.  Old events were 0; read change was 0 (none); write change was 1 (add): Bad file descriptor
[warn] Epoll ADD(4) on fd 86 failed.  Old events were 0; read change was 0 (none); write change was 1 (add): Bad file descriptor
[warn] Epoll ADD(4) on fd 88 failed.  Old events were 0; read change was 0 (none); write change was 1 (add): Bad file descriptor
[warn] Epoll ADD(4) on fd 91 failed.  Old events were 0; read change was 0 (none); write change w

In [ ]:
# process include master_process 5, 9, 17 processes
# N = 1000
! mpiexec --allow-run-as-root -np 5 python mpi-mul.py

In [ ]:
# process include master_process 5, 9, 17 processes
# N = 1000
! mpiexec --allow-run-as-root -np 9 python mpi-mul.py

In [ ]:
# process include master_process 5, 9, 17 processes
# N = 1000
! mpiexec --allow-run-as-root -np 17 python mpi-mul.py

# FILE : mpi-mul2.py

Cannon's algorithm แต่คำตอบมันผิดแล้วมัน run นานมากผมไม่รู้จะทำยังไงแล้วครับ

ตัวแปร

**Line 11 :** workers คือ slave processes = comm.Get_size() - 1

**Line 14 :** N คือขนาดของ array NxN numpy array

**Line 17 :** mtrx_A คือ first numpy array value between 0 - 9 size NxN

**Line 18 :** mtrx_B คือ second numpy array value between 0 - 9 size NxN

**Line 19 :** mtrx_C คือ result numpy array

**Line 22 :** mul_matrix function คือ matrix multiplier function ทำ *Z[i][j] += X[i][j] x Y[i][j]* ในทุก slave process

**Line 30 :** distribute_data function คือแยก data ของ matrix จาก master process เป็นจำนวนเท่า square root ของ workers และยัดเข้าไปใน slave process แต่ละตัว โดยใช้ comm.send()

**Line 54 :** assemble_data function คือรวม data ของ matrix ที่ผ่านการคูณมาแล้วใน slave processes มาใส่ใน mtrx_C โดยใช้ comm.recv ใน master process

**Line 60 :** slave process roll ให้ matrix ที่ได้มา shift left matrix A แถวที่ i ไป i ช่อง shift up matrix B หลักที่ i ไป i ช่อง และค่อยทำการคูณ matrix และหลังจากนั้น shift left matrix A ทุกแถวไป 1 ช่อง และshift up matrix B ทุกหลักไป 1 ช่อง และวนไปเรื่อยๆ

In [ ]:
%%writefile mpi-mul2.py
from mpi4py import MPI
from random import randint
import numpy as np
import time
import math

comm = MPI.COMM_WORLD
rank = comm.Get_rank()
workers = comm.Get_size() - 1

N = 10

mtrx_A = np.random.randint(10,size=(N,N))
mtrx_B = np.random.randint(10,size=(N,N))
mtrx_C = np.zeros((N,N))

def mul_matrix(X, Y,i,j):
    Z = X[i][j] * Y[i][j]
    return Z


def distibute_data():
    def split_matrix(seq, p):
        rows = []
        n = int(len(seq) / math.sqrt(p))
        r = int(len(seq) % math.sqrt(p))
        b, e = 0, n + min(1, r)
        for i in range(int(math.sqrt(p))):
            rows.append(seq[b:e])
            r = max(0, r - 1)
            b, e = e, e + n + min(1, r)

        return rows

    rows = split_matrix(mtrx_A, workers)

    pid = 1
    for row in rows:
        comm.send(row, dest=pid, tag=1)
        comm.send(mtrx_B, dest=pid, tag=2)
        pid = pid + 1


def assemble_data():
    global mtrx_C

    pid = 1
    for n in range(int(math.sqrt(workers))):
        row = comm.recv(source=pid, tag=pid)
        mtrx_C = mtrx_C + row
        pid = pid + 1


def master_process():
    distibute_data()
    assemble_data()


def slave_process():
    # receive data from master node
    x = comm.recv(source=0, tag=1)
    y = comm.recv(source=0, tag=2)
    z = np.zeros((N,N))

    x2 = [np.roll(np.array(i), -1) for i in x]
    y2 =  np.array(y)
    for i in range(len(y2)):
      y2[:,i] = np.roll(y2[:,i], -i)
    # multiply the received matrix and send the result back to master

    for i in range(len(x2)):
      for j in range(len(y2)):
        z[i][j] = mul_matrix(x2, y2,i,j)
        np.roll(x2,-1,axis=0)
        np.roll(y2,-1,axis=1)

    comm.send(z, dest=0, tag=rank)


if __name__ == '__main__':
    if rank == 0:

        # start time
        t1 = time.time()

        master_process()

        # end time
        t2 = time.time()

        print(mtrx_A)
        print(mtrx_B)
        print(mtrx_C)
        print('Time taken in seconds', t2 - t1)
    else:
        slave_process()

Overwriting mpi-mul2.py


In [ ]:
! mpiexec --allow-run-as-root -np 4 python mpi-mul2.py

[[8 7 4 8 1 3 6 6 4 8]
 [3 1 2 0 3 2 1 4 4 2]
 [7 2 1 8 2 3 1 0 6 1]
 [5 5 5 1 1 3 1 7 0 9]
 [4 3 8 6 0 1 4 0 4 7]
 [5 6 0 1 8 8 2 5 7 7]
 [3 7 4 7 2 0 9 2 0 4]
 [1 9 3 4 2 3 3 7 1 6]
 [8 4 0 0 4 2 5 8 4 9]
 [8 9 8 2 9 8 4 4 6 8]]
[[4 1 1 1 9 2 2 1 4 2]
 [6 3 7 1 4 8 9 8 2 0]
 [9 1 2 7 7 7 6 6 0 6]
 [2 4 9 4 7 4 1 7 9 7]
 [4 6 6 1 8 1 4 8 0 2]
 [2 1 5 9 3 0 4 9 5 0]
 [9 9 8 3 8 5 1 7 8 8]
 [7 1 8 3 6 8 9 5 7 9]
 [1 1 5 4 3 8 0 0 2 8]
 [0 7 7 5 1 4 4 1 7 4]]
[[28. 12. 16.  4. 24.  0.  6. 20. 16. 32.]
 [ 6.  2.  0.  3.  6.  5. 36.  0. 14.  6.]
 [18.  4. 48. 18. 24.  8.  0.  6.  4.  0.]
 [10. 30.  5.  3. 18.  8. 28.  0. 18. 30.]
 [12.  8. 48.  0.  3. 16.  0. 32.  0. 28.]
 [12.  0.  8. 32.  8.  4. 45. 42. 63. 10.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]]
Time taken in seconds 0.472139835357666
